# Experiment 2:

- Our base sketch transformer architecture + class embedding layer
- Multi-class Conditional Model
- DeltaPenPositionTokenizer

In [26]:
from experiment_dir import set_cwd_project_root

set_cwd_project_root()

In [ ]:
from dataset import QuickDrawDataset
from sketch_tokenizers import DeltaPenPositionTokenizer
from models import SketchTransformerConditional
from runner import SketchTrainer, sample
from prepare_data import stroke_to_bezier_single, clean_svg

label_names = ["bird", "crab", "guitar"]
dataset = QuickDrawDataset(label_names=label_names, download=True)
tokenizer = DeltaPenPositionTokenizer(bins=32)

model = SketchTransformerConditional(
    vocab_size=len(tokenizer.vocab),
    d_model=512,
    nhead=8,
    num_layers=8,
    max_len=200,
    num_classes=len(label_names),
)

training_config = {
    "batch_size": 128,
    "num_epochs": 15,
    "learning_rate": 1e-4,
    "log_dir": "logs/sketch_transformer_experiment_2",
    "splits": [0.85, 0.1, 0.05],
    # "use_padding_mask": True,
}

trainer = SketchTrainer(model, dataset, tokenizer, training_config)

Loading QuickDrawDataset: 3it [00:12,  4.01s/it]
c:\Users\matth\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
Tokenizing dataset: 100%|██████████| 3/3 [01:44<00:00, 34.94s/it]


No checkpoints found, starting from scratch.


Initial Eval: 100%|██████████| 258/258 [00:01<00:00, 196.73it/s]


In [6]:
trainer.train_mixed(training_config["num_epochs"])

Epoch 1/15 [val]: 100%|██████████| 258/258 [00:12<00:00, 21.10it/s]


Epoch 1 | Train Loss: 1.2621 | Val Loss: 1.1164


Epoch 2/15 [val]: 100%|██████████| 258/258 [00:12<00:00, 21.34it/s]


Epoch 2 | Train Loss: 1.1201 | Val Loss: 1.0720


Epoch 3/15 [val]: 100%|██████████| 258/258 [00:12<00:00, 21.50it/s]


...

Epoch 10 | Train Loss: 0.9926 | Val Loss: 0.9826


Epoch 11/15 [val]: 100%|██████████| 258/258 [00:12<00:00, 21.02it/s]


Epoch 11 | Train Loss: 0.9878 | Val Loss: 0.9794


Epoch 12/15 [train]:   7%|▋         | 146/2191 [00:22<05:16,  6.45it/s]


KeyboardInterrupt: 

In [ ]:
generations_inline = ""
generations = []

for j, label_name in enumerate(label_names):
    for i in range(5):
        generated = sample(
            model=trainer.model,
            start_tokens=[trainer.tokenizer.vocab["START"]],
            temperature=0.8,
            top_k=20,
            top_p=0.7,
            greedy=False,
            eos_id=trainer.tokenizer.vocab["END"],
            class_label=j,
        )
        decoded_sketch = tokenizer.decode(generated, stroke_width=0.3)
        decoded_sketch = stroke_to_bezier_single(decoded_sketch)
        decoded_sketch = clean_svg(decoded_sketch)

        generations_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Generated {label_name}</b><br>{decoded_sketch}</div>'
        generations.append((generated, decoded_sketch))
        
        # import time
        
        # with open(f"save_{label_name}_{i}_{int(time.time())}.svg", "w") as f:
        #     f.write(decoded_sketch)


from IPython.display import HTML, display

display(HTML(generations_inline))